In [1]:
from requests import get, post
from web3 import Web3, HTTPProvider
import web3
from icecream import ic
import numpy as np
import pandas as pd
import sys
from dotenv import load_dotenv
import os
import math
import json

In [2]:
# Add the directory containing the module to the Python path
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Import functions from the module
from arcadia_analysis.uniV3Pricing import get_account_data, get_uniswap_slot0, get_nft_positions_details, get_amounts_from_ticks

In [3]:
# Explicitly specify the path to .env file using forward slashes
load_dotenv(dotenv_path='.env')

# Create the Web3 provider using the Alchemy URL
w3 = Web3(HTTPProvider(os.getenv('BLOCKPI_BASE'))) # BASE

# Check connection
print(f"Web3 is connected: {w3.is_connected()}")


Web3 is connected: True


In [4]:
def sqrt_price_x96_to_price(sqrt_price_x96):
    Q96 = 0x1000000000000000000000000
    return (sqrt_price_x96 / Q96) ** 2


def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))


def get_slot0_info(pool_address, w3):
    abi_uniswap = [
        {
            "inputs": [],
            "name": "slot0",
            "outputs": [
                {"internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
                {"internalType": "int24", "name": "tick", "type": "int24"},
                {"internalType": "uint16", "name": "observationIndex", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinality", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinalityNext", "type": "uint16"},
                {"internalType": "uint8", "name": "feeProtocol", "type": "uint8"},
                {"internalType": "bool", "name": "unlocked", "type": "bool"},
            ],
            "stateMutability": "view",
            "type": "function",
        }
    ]
    
    abi_aerodrome = [
        {
            "inputs": [],
            "name": "slot0",
            "outputs": [
                {"internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
                {"internalType": "int24", "name": "tick", "type": "int24"},
                {"internalType": "uint16", "name": "observationIndex", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinality", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinalityNext", "type": "uint16"},
                {"internalType": "bool", "name": "unlocked", "type": "bool"},
            ],
            "stateMutability": "view",
            "type": "function",
        }
    ]
    
    contract_address = Web3.to_checksum_address(pool_address)
    
    # Try to use the Uniswap ABI
    try:
        contract = w3.eth.contract(address=contract_address, abi=abi_uniswap)
        result = contract.functions.slot0().call()
        return {
            "sqrtPriceX96": result[0],
            "tick": result[1]
        }
    except (web3.exceptions.ContractLogicError, web3.exceptions.BadFunctionCallOutput):
        # If Uniswap ABI fails, use the Aerodrome ABI
        try:
            contract = w3.eth.contract(address=contract_address, abi=abi_aerodrome)
            result = contract.functions.slot0().call()
            return {
                "sqrtPriceX96": result[0],
                "tick": result[1]
            }
        except (web3.exceptions.ContractLogicError, web3.exceptions.BadFunctionCallOutput):
            return None



In [48]:
from collections import defaultdict

# Function to add or update the dictionary
def add_or_update_dict(dictionary, key, value):
    dictionary[key] += value
    
def get_arcadia_account_nft_position(asset_data, w3):
    """
    Sample asset_data:
    [
        ["0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22", 
        "0x4200000000000000000000000000000000000006", 
        "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1"], 
        [0, 0, 105363], 
        [12289573077240346, 13102161708949451, 1]
    ]
    """
    POOL_NFT_MAPPINGS = [
        # Uniswap v3 Pools
        {
            "name": "wETH-USDC",
            "pool": "0xd0b53D9277642d899DF5C87A3966A349A798F224",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "decimal0": 18,
            "decimal1": 6
        },
        {
            "name": "wETH-USDbC",
            "pool": "0x4C36388bE6F416A29C8d8Eee81C771cE6bE14B18",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 18,
            "decimal1": 6
        },
        {
            "name": "wETH-wstETH",
            "pool": "0x20E068D76f9E90b90604500B84c7e19dCB923e7e",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "cbETH-wETH",
            "pool": "0x10648BA41B8565907Cfa1496765fA4D95390aa0d",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22",
            "token1": "0x4200000000000000000000000000000000000006",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "USDC-USDbC",
            "pool": "0x06959273E9A65433De71F5A452D529544E07dDD0",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 6,
            "decimal1": 6
        },
        {
            "name": "DAI-USDbC",
            "pool": "0x22F9623817F152148B4E080E98Af66FBE9C5AdF8",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 18,
            "decimal1": 6
        },
        
        
        # Aerodrome Pools
        {
            "name": "wETH-wstETH",
            "pool": "0x861A2922bE165a5Bd41b1E482B49216b465e1B5F",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "cbETH-wETH",
            "pool": "0x47cA96Ea59C13F72745928887f84C9F52C3D7348",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22",
            "token1": "0x4200000000000000000000000000000000000006",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "USDC-USDbC",
            "pool": "0x98c7A2338336d2d354663246F64676009c7bDa97",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 6,
            "decimal1": 6
        },
        {
            "name": "wETH-USDC",
            "pool": "0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "decimal0": 18,
            "decimal1": 6
        }
    ]
    
    zero_indices = [index for index, value in enumerate(asset_data[1]) if value == 0]
    non_zero_indices = [index for index, value in enumerate(asset_data[1]) if value != 0]

    result = defaultdict(int)

    # Creating a lookup dictionary for quick access with normalized tokens
    pool_lookup = {}
    for pool in POOL_NFT_MAPPINGS:
        nft = pool["nft"].lower()
        token0 = pool["token0"].lower()
        token1 = pool["token1"].lower()
        
        pool_lookup[(nft, token0, token1)] = pool

    for i in zero_indices:
        add_or_update_dict(result, asset_data[0][i], asset_data[2][i])
    
    for i in non_zero_indices:
        nft_contract = asset_data[0][i].lower()
        nft_positions_details = get_nft_positions_details(nft_contract_address=nft_contract, w3=w3, token_id=asset_data[1][i])
        
        ic(nft_positions_details)
        
        if not nft_positions_details:
            add_or_update_dict(result, nft_contract, 0) # Records the NFT if it doesnot belong to Uniswap
            continue  # Skip if nft_positions_details could not be fetched
        
        # Extracting and normalizing token0 and token1 from nft_positions_details
        token0 = nft_positions_details["token0"].lower()
        token1 = nft_positions_details["token1"].lower()

        # Finding the matching pool using the lookup dictionary
        matching_pool = pool_lookup.get((nft_contract, token0, token1))
        ic(matching_pool)
        if not matching_pool:
            add_or_update_dict(result, nft_contract, 0) # Records the NFT if it doesnot belong to the Mapping
            continue  # Skip if no matching pool is found
        
        slot0 = get_slot0_info(pool_address=matching_pool["pool"], w3=w3) 
        ic(slot0)
        if not slot0:
            continue  # Skip if slot0 details could not be fetched
        
        current_tick = slot0["tick"]
        lower_tick = nft_positions_details["tickLower"]
        upper_tick = nft_positions_details["tickUpper"]
        
        amount0, amount1 = get_amounts_from_ticks(current_tick, lower_tick, upper_tick, nft_positions_details["liquidity"])
        
        ic(amount0, amount1)
        
        add_or_update_dict(result, nft_positions_details["token0"], amount0)
        add_or_update_dict(result, nft_positions_details["token1"], amount1)
    
    return result

In [50]:
account_data_test = get_account_data("0x65ae12225af4972eb600195e9af5e52cf6cc9aec",w3)
print(json.dumps(account_data_test, indent=4))

position_distribution = get_arcadia_account_nft_position(account_data_test, w3)
print(json.dumps(position_distribution, indent=4))

ic| nft_positions_details: {'fee': 100,
                            'feeGrowthInside0LastX128': 0,
                            'feeGrowthInside1LastX128': 0,
                            'liquidity': 10035761376763239199,
                            'nonce': 0,
                            'operator': '0x0000000000000000000000000000000000000000',
                            'tickLower': -1631,
                            'tickUpper': -1530,
                            'token0': '0x4200000000000000000000000000000000000006',
                            'token1': '0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452',
                            'tokensOwed0': 0,
                            'tokensOwed1'

[
    [
        "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22",
        "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
        "0xB6fe221Fe9EeF5aBa221c348bA20A1Bf5e73624c",
        "0x4200000000000000000000000000000000000006",
        "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
        "0x9f42361B7602Df1A8Ae28Bf63E6cb1883CD44C27"
    ],
    [
        0,
        0,
        0,
        0,
        684487,
        31
    ],
    [
        209769491377690746,
        689840961176182,
        217551170863881984,
        1872086218213525,
        1,
        1
    ]
]


: 0}
ic| matching_pool: {'decimal0': 18,
                    'decimal1': 18,
                    'name': 'wETH-wstETH',
                    'nft': '0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1',
                    'pool': '0x20E068D76f9E90b90604500B84c7e19dCB923e7e',
                    'token0': '0x4200000000000000000000000000000000000006',
                    'token1': '0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452'}
ic| slot0: {'sqrtPriceX96': 73167814878362488983394436265, 'tick': -1592}
ic| amount0: 33634538252725018, amount1: 18053930423268471
ic| nft_positions_details: None


{
    "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22": 209769491377690746,
    "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452": 18743771384444653,
    "0xB6fe221Fe9EeF5aBa221c348bA20A1Bf5e73624c": 217551170863881984,
    "0x4200000000000000000000000000000000000006": 35506624470938543,
    "0x9f42361b7602df1a8ae28bf63e6cb1883cd44c27": 0
}


In [47]:
nft_pos = get_nft_positions_details('0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1', w3, 91470)
print(json.dumps(nft_pos, indent=4))

{
    "nonce": 0,
    "operator": "0x0000000000000000000000000000000000000000",
    "token0": "0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb",
    "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
    "fee": 100,
    "tickLower": -276336,
    "tickUpper": -276315,
    "liquidity": 149899570935539233,
    "feeGrowthInside0LastX128": 21550026498249782294602674559923957808042,
    "feeGrowthInside1LastX128": 22702457499150940817188780766,
    "tokensOwed0": 0,
    "tokensOwed1": 0
}
